In [1]:
print("Ok")

Ok


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [5]:
PINECONE_API_KEY = "77f3280e-ab97-41f0-815b-260b12cbdde4"
PINECONE_API_ENV = "gcp-starter"

In [6]:
#Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf("data/")

In [8]:
# create text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
#each chunk 500 tokens, total chunks = 7020
text_chunks = text_split(extracted_data)
print("length of chunk:", len(text_chunks))

length of chunk: 7020


In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

In [9]:
#embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [12]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [ ]:
#pip install --upgrade pinecone-client
#pip install --upgrade langchain-pinecone
#pip install pinecone-client

In [13]:
import os
os.environ["PINECONE_API_KEY"]="77f3280e-ab97-41f0-815b-260b12cbdde4"

In [14]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '77f3280e-ab97-41f0-815b-260b12cbdde4')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [15]:
import pinecone

In [16]:
from langchain_pinecone import PineconeVectorStore

In [17]:
import os
os.environ["PINECONE_API_KEY"]="77f3280e-ab97-41f0-815b-260b12cbdde4"

In [18]:
index_name = "medical-chatbot"

In [19]:
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [20]:
#docsearch = Pinecone.from_existing_index(index_name,embeddings)
query="What is ChickenPox"
docs = docsearch.similarity_search(query, k=3)

print("Result",docs)

Result [Document(page_content='drugs, health care providers should inquire whether thepatient has any of the following conditions:\n•chickenpox or recent exposure to someone with chick-\nenpox\n• shingles (Herpes zoster)\n• mouth sores\n• current or past seizures\nGALE ENCYCLOPEDIA OF MEDICINE 2 247Anticancer drugs\nAnti Cancer Drugs\nGeneric (Brand Name) Clinical Uses Common Side Effects To Drug\nAltretamine (Hexalen) Treatment of advanced ovarian cancer Bone marrow depression, nausea and vomiting', metadata={'page': 260.0, 'source': 'data\\Medical_book.pdf'}), Document(page_content='drugs, health care providers should inquire whether thepatient has any of the following conditions:\n•chickenpox or recent exposure to someone with chick-\nenpox\n• shingles (Herpes zoster)\n• mouth sores\n• current or past seizures\nGALE ENCYCLOPEDIA OF MEDICINE 2 247Anticancer drugs\nAnti Cancer Drugs\nGeneric (Brand Name) Clinical Uses Common Side Effects To Drug\nAltretamine (Hexalen) Treatment of adv

In [21]:
prompt_template = """
Use the following pieces of information to answeer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [22]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [27]:
#pip install ctransformers

Note: you may need to restart the kernel to use updated packages.


In [29]:
from ctransformers.langchain import CTransformers

In [2]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained('model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama',lib='avx')                       'temperature':0.8})

SyntaxError: invalid syntax (3378998946.py, line 3)

In [30]:

llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

OSError: [WinError -529697949] Windows Error 0xe06d7363

In [27]:
from CTransformers import CTransformers

# Provide the correct path to your model file
model_path = "model/llama-2-7b-chat.ggmlv3.q4_0.bin"

# Specify the model type and configuration
model_config = {
    'max_new_tokens': 512,
    'temperature': 0.8
}

try:
    llm = CTransformers(model=model_path, model_type="llama", config=model_config)
    print("Model loaded successfully!")
except OSError as e:
    print(f"Error loading the model: {e}")


ModuleNotFoundError: No module named 'CTransformers'